In [3]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

client = OpenAI(
    api_key=os.getenv("MOONSHOT_API_KEY"),
    base_url="https://api.moonshot.cn/v1",
)

completion = client.chat.completions.create(
    model = "kimi-k2-turbo-preview",
    messages = [
        {"role": "system", "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。"},
        {"role": "user", "content": "你好，我叫李雷，1+1等于多少？"}
    ],
    temperature = 0.6,
)

print(completion.choices[0].message.content)

你好，李雷！  
1 + 1 = 2


In [5]:
# 实验1: 无历史 vs 有历史
def experiment_1():
    # 场景: 用户问"他是谁?"

    # Case A: 无上下文
    response_a = client.chat.completions.create(
        model = "kimi-k2-turbo-preview",
        messages=[{"role": "user", "content": "他是谁?"}]
    )

    # Case B: 有上下文(之前提到了"张三")
    response_b = client.chat.completions.create(
        model = "kimi-k2-turbo-preview",
        messages=[
            {"role": "user", "content": "我有个朋友叫张三"},
            {"role": "assistant", "content": "好的,你有个朋友叫张三"},
            {"role": "user", "content": "他是谁?"}
        ]
    )

    print("无上下文:", response_a.choices[0].message.content)
    print("有上下文:", response_b.choices[0].message.content)
    # 预期: 无上下文会说"我不知道",有上下文会说"张三"

# 运行实验
print("=== 实验1: 验证上下文的作用 ===")
experiment_1()

=== 实验1: 验证上下文的作用 ===
无上下文: 你说的“他”是指谁呢？可以再具体一点吗？比如你是在问某个历史人物、明星、小说角色，还是你刚刚提到的某个人？给我一点线索，我就能帮你找出答案。
有上下文: 你刚才说“我有个朋友叫张三”，所以我只知道他叫张三，是你朋友；至于他到底是谁、做什么、性格如何，目前你还没告诉我任何其他信息。如果你愿意，可以再多说一点，我就能帮你一起“拼”出他是谁。


In [6]:
# 实验2: 短历史 vs 长历史
def experiment_2():
    print("=== 实验2: 验证Context Rot (上下文衰减) ===")
    
    # 生成50轮无关对话 (减少轮数避免超出Kimi的上下文长度限制)
    long_history = []
    for i in range(50):
        long_history.append({"role": "user", "content": f"今天天气{i}号"})
        long_history.append({"role": "assistant", "content": f"好的{i}号"})

    # 在最开始提到"我喜欢Python"
    long_history.insert(0, {"role": "user", "content": "我喜欢Python"})
    long_history.insert(1, {"role": "assistant", "content": "好的,你喜欢Python"})

    # 现在问"我喜欢什么?"
    long_history.append({"role": "user", "content": "我喜欢什么编程语言?"})

    response = client.chat.completions.create(
        model="kimi-k2-turbo-preview",
        messages=long_history
    )

    print("长历史后的回答:", response.choices[0].message.content)
    print("\n预期分析:")
    print("- 如果Kimi记得Python，说明上下文保持能力强")
    print("- 如果Kimi不记得，说明存在context rot现象")
    
    # 对比实验：短历史
    print("\n--- 对比：短历史 ---")
    short_history = [
        {"role": "user", "content": "我喜欢Python"},
        {"role": "assistant", "content": "好的,你喜欢Python"},
        {"role": "user", "content": "我喜欢什么编程语言?"}
    ]
    
    response_short = client.chat.completions.create(
        model="kimi-k2-turbo-preview",
        messages=short_history
    )
    print("短历史后的回答:", response_short.choices[0].message.content)

# 运行实验
experiment_2()

=== 实验2: 验证Context Rot (上下文衰减) ===
长历史后的回答: 你之前说过“我喜欢Python”，所以答案是：Python。

预期分析:
- 如果Kimi记得Python，说明上下文保持能力强
- 如果Kimi不记得，说明存在context rot现象

--- 对比：短历史 ---
短历史后的回答: 你刚刚说“我喜欢Python”，所以答案就是：Python。
